In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# Read in CSV file
regional_df = pd.read_csv("Table_CSVs/regional.csv")
regional_df

,DOEID,REGIONC,DIVISION,state_postal,BA_climate,TOTALBTU,TOTALDOL
0,100001,WEST,Mountain South,NM,Mixed-Dry,144647.71,2656.89
1,100002,SOUTH,West South Central,AR,Mixed-Humid,28034.61,975.00
2,100003,WEST,Mountain South,NM,Mixed-Dry,30749.71,522.65
3,100004,SOUTH,South Atlantic,SC,Mixed-Humid,86765.19,2061.77
4,100005,NORTHEAST,Middle Atlantic,NJ,Mixed-Humid,59126.93,1463.04
...,...,...,...,...,...,...,...
18491,118492,SOUTH,South Atlantic,MD,Mixed-Humid,49930.49,1098.51
18492,118493,NORTHEAST,New England,ME,Very-Cold,222186.04,3613.44
18493,118494,SOUTH,West South Central,TX,Hot-Humid,51593.72,1428.31
18494,118495,SOUTH,South Atlantic,SC,Hot-Humid,63555.21,2224.94


In [3]:
# Create DataFrame with regional information
regional_df = regional_df.drop(columns=['DOEID', 'TOTALDOL'])
regional_df.head()

,REGIONC,DIVISION,state_postal,BA_climate,TOTALBTU
0,WEST,Mountain South,NM,Mixed-Dry,144647.71
1,SOUTH,West South Central,AR,Mixed-Humid,28034.61
2,WEST,Mountain South,NM,Mixed-Dry,30749.71
3,SOUTH,South Atlantic,SC,Mixed-Humid,86765.19
4,NORTHEAST,Middle Atlantic,NJ,Mixed-Humid,59126.93


In [4]:
#Checking nulls
regional_df.isnull().sum()

REGIONC         0
DIVISION        0
state_postal    0
BA_climate      0
TOTALBTU        0
dtype: int64

In [5]:
# Define a function to perform binning on TOTALBTU column
def bin_total_btu(total_btu):
    if total_btu < 55000:
        return 'Low'
    elif total_btu >= 55000 and total_btu < 95000:
        return 'Medium'
    else:
        return 'High'

In [7]:
# Apply binning function to create a new column 'BTU_Bin'
regional_df['BTU_Bin'] = regional_df['TOTALBTU'].apply(bin_total_btu)

In [8]:
# Check the balance of bins
bin_counts = regional_df['BTU_Bin'].value_counts()

In [9]:
# Display the counts of samples in each bin
print("Counts of samples in each bin:")
print(bin_counts)

Counts of samples in each bin:
BTU_Bin
High      6240
Low       6194
Medium    6062
Name: count, dtype: int64


In [10]:
# Perform one-hot encoding on the 'BTU_Bin' column
X_encoded = pd.get_dummies(regional_df.drop(['TOTALBTU'], axis=1), drop_first=True)
y = regional_df['TOTALBTU']

In [21]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.05, random_state=42)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Initialize the Linear Regression model
lr_model = LinearRegression()

# Train the model
lr_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = lr_model.predict(X_test_scaled)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 567474693.1939898
R-squared (R2): 0.7529492375445945
